<a href="https://colab.research.google.com/github/killianb22/BI_CA/blob/main/APA_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re
import string
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter
import pandas as pd
from translate import Translator

In [3]:
twitter_data = pd.read_csv("/content/Twitter_Data.csv")

In [4]:
print(twitter_data.shape)

(162980, 2)


In [5]:
twitter_data.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [6]:
# Find the number of positive and negative reviews
print('Number of positive and negative reviews: ', twitter_data.category.value_counts())

# Find the proportion of positive and negative reviews
print('Proportion of positive and negative reviews: ', twitter_data.category.value_counts() / len(twitter_data))

Number of positive and negative reviews:   1.0    72250
 0.0    55213
-1.0    35510
Name: category, dtype: int64
Proportion of positive and negative reviews:   1.0    0.443306
 0.0    0.338772
-1.0    0.217879
Name: category, dtype: float64


In [7]:
# Define a dictionary for the six lexical fields related to inflation in French, English, and Turkish
lexical_fields = {
    'inflation_economics': ['inflation', 'hyperinflation', 'deflation', 'stagflation', 'price index',
                            'monetary policy', 'purchasing power', 'enflasyon', 'devalüasyon',
                            'fiyat endeksi', 'para politikası', 'alım gücü', 'inflation', 'déflation',
                            'pouvoir d\'achat', 'politique monétaire', 'indice des prix'],
    'expensive': ['expensive', 'high-priced', 'costly', 'overpriced', 'pricey',
                  'pahalı', 'maliyetli', 'yüksek fiyatlı', 'cher', 'coûteux', 'hors de prix'],
    'cheap': ['cheap', 'low-cost', 'budget', 'affordable', 'inexpensive',
              'ucuz', 'düşük maliyetli', 'ekonomik', 'pas cher', 'abordable', 'bon marché'],
    'prices_costs': ['price', 'cost', 'expense', 'fee', 'charge', 'rate', 'tariff',
                     'fiyat', 'maliyet', 'ücret', 'tarife', 'prix', 'coût', 'frais', 'tarif'],
    'statistical_institutions': ['Bureau of Labor Statistics', 'Consumer Price Index', 'Federal Reserve',
                                  'Eurostat', 'TÜİK', 'Merkez Bankası', 'İstatistik Kurumu',
                                  'Insee', 'Banque centrale', 'Office de statistique'],
    'additional_keywords': ['economy', 'market', 'value', 'money', 'finance',
                            'ekonomi', 'piyasa', 'değer', 'para', 'finans',
                            'économie', 'marché', 'valeur', 'argent', 'finance']
}

In [9]:
# Function to check if a tweet contains any of the keywords from the lexical fields
def filter_tweets(tweet, fields):
    if isinstance(tweet, str):
        return any(any(keyword in tweet.lower() for keyword in field) for field in fields.values())
    else:
        # If tweet is not a string, we do not consider it for our lexical fields
        return False

In [25]:
import re
# Apply the adjusted filter function to the DataFrame
filtered_df = twitter_data[twitter_data['clean_text'].apply(lambda tweet: filter_tweets(tweet, lexical_fields))]

# Function to identify invalid characters
# We define invalid characters as anything that is NOT alphanumeric, standard punctuation, or whitespace
def contains_invalid_characters(text):
    # Define a set of allowed punctuation characters
    allowed_punctuation = "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
    pattern = r'^[A-Za-z0-9\s' + re.escape(allowed_punctuation) + r']+$'
    return not re.match(pattern, text)

# Apply the function to the 'clean_text' column to create a mask for rows with invalid characters
invalid_mask = filtered_df['clean_text'].apply(lambda x: contains_invalid_characters(str(x)))

In [26]:
# Invert the mask to filter out the invalid rows
clean_df = filtered_df[~invalid_mask]

# Now 'clean_df' contains only rows with valid characters
print(f"Number of rows after removing invalid characters: {len(clean_df)}")

Number of rows after removing invalid characters: 14552


In [28]:
# Display the number of tweets before and after filtering/cleaning
original_count = twitter_data.shape[0]
filtered_count = clean_df.shape[0]
print(f'Original tweet count: {original_count}')
print(f'Filtered tweet count: {filtered_count}')
print(clean_df.head())

Original tweet count: 162980
Filtered tweet count: 14552
                                           clean_text  category
25  \ndevelopment has become mass movement under m...       1.0
32  modi govts slashing indias education budget cl...      -1.0
36  check out latest article premier archery leagu...       1.0
43  yes good job highly insensitivearrogant incomp...       1.0
48  with welfare delivery gst ibc and feo place mo...       1.0


In [29]:
# Install the library and functions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize


stop = stopwords.words('english')
excluding = ['against', 'not', 'don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't",
             'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't",
             'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
             'needn', "needn't",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren',
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]


stop_words = [word for word in stop if word not in excluding]
snow = SnowballStemmer('english')

def process_text(texts):
    final_text_list=[]
    for sent in texts:
        if isinstance(sent, str) == False:
            sent = ""

        filtered_sentence=[]

        sent = sent.lower()
        sent = sent.strip()
        sent = re.sub('\s+', ' ', sent)
        sent = re.compile('<.*?>').sub('', sent)

        for w in word_tokenize(sent):
            if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words):
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence)
        final_text_list.append(final_string)

    return final_text_list

In [32]:
from sklearn.model_selection import train_test_split
# Splitting the filtered data into training and testing sets with an 80:20 split
train_df, test_df = train_test_split(clean_df, test_size=0.2, random_state=42)

In [33]:
# Displaying the number of entries in the training and testing sets
print(f"Number of entries in the training set: {len(train_df)}")
print(f"Number of entries in the testing set: {len(test_df)}")

Number of entries in the training set: 11641
Number of entries in the testing set: 2911


In [34]:
print("Processing the text field")
train_text_list = process_text(train_df["clean_text"].tolist())
test_text_list = process_text(test_df["clean_text"].tolist())

y_train= train_df["category"].tolist()
y_test= test_df["category"].tolist()

Processing the text field


In [21]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
import gensim
from gensim.models import Word2Vec

pipeline = Pipeline([
    ('text_vect', CountVectorizer(binary=True,
                                  max_features=200)),
    ('knn', KNeighborsClassifier())
                                ])
from sklearn import set_config
set_config(display='diagram')
pipeline

Pipeline(steps=[('text_vect', CountVectorizer(binary=True, max_features=200)),
                ('knn', KNeighborsClassifier())])

In [22]:
# We using lists of processed text fields
X_train = train_text_list
X_test = test_text_list

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('text_vect', CountVectorizer(binary=True, max_features=200)),
                ('knn', KNeighborsClassifier())])

In [35]:
# Implement this
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_test)
print(confusion_matrix(y_test, val_predictions))
print(classification_report(y_test, val_predictions))
print("Accuracy (validation):", accuracy_score(y_test, val_predictions))

ValueError: ignored